# Recruitment Case Study

The outcome of this technical assessment is to create two models that can be used by analysts to derive value and answers from the questions that they have. 

<b>Please note</b> that the code in this notebook does not include a sophisticated logging strategy as the strategy used within the business environment would need to be applied before this is put into production. Jupyter Notebook was used for this as it is a user-friendly option of explaining the thought process throughout the assessment.

## Install Libraries

In [1]:
#!pip install openpyxl 

## Import Libraries

In [2]:
import pandas as pd
import numpy as np

## Read Input Datasets

Extract raw data from the csv files that have been provided by Yoco.

In [3]:
# Change the base path to point to where the csv files are stored
file_path = 'Data'

# Read csv files from the base file path
df_application = pd.read_csv(r'%s\application.csv' % file_path)
df_department = pd.read_csv(r'%s\department.csv' % file_path)
df_interview = pd.read_csv(r'%s\interview.csv' % file_path)
df_job = pd.read_csv(r'%s\job.csv' % file_path)
df_job_application = pd.read_csv(r'%s\job_application.csv' % file_path)
df_job_department = pd.read_csv(r'%s\job_department.csv' % file_path)
df_job_post = pd.read_csv(r'%s\job_post.csv' % file_path)
df_job_stage = pd.read_csv(r'%s\job_stage.csv' % file_path)
df_scheduled_interview = pd.read_csv(r'%s\scheduled_interview.csv' % file_path)
df_scheduled_interviewer = pd.read_csv(r'%s\scheduled_interviewer.csv' % file_path)

# List of datasets
dataset_names = ["df_application", "df_department", "df_interview", "df_job", "df_job_application", "df_job_department", "df_job_post", "df_job_stage", "df_scheduled_interview", "df_scheduled_interviewer"]
datasets = [df_application, df_department, df_interview, df_job, df_job_application, df_job_department, df_job_post, df_job_stage, df_scheduled_interview, df_scheduled_interviewer]

## Data Analysis

Analyse the data to understand the relationship between the datasets.

### Understanding the Data Structure: All Columns (per Dataset)
By viewing all columns in each dataset, an understanding of the existing raw data structure maybe be developed

In [4]:
# Create dataframe to store dataset column information
df_dataset_columns = pd.DataFrame(columns={'Dataset','Number of Columns', 'Columns', 'Number of Records'})

# Show all columns of each dataframe
for index, dataset in enumerate(datasets):
    
    # Test Harness: Print dataset names and columns of each dataset
    #print(dataset_names[index])
    #print(dataset.columns)
    
    # Append a row for each dataset with number of columns
    df_dataset_columns = df_dataset_columns.append({'Dataset':dataset_names[index],'Number of Columns':len(dataset.columns), 'Columns':str(dataset.columns), 'Number of Records': len(dataset)}, ignore_index=True)

# Display the table of columns per dataset in descending order (by Number of Columns)
df_dataset_columns.sort_values(by=['Number of Columns'], ascending=False)

,Columns,Number of Columns,Dataset,Number of Records
0,"Index(['id', 'candidate_id', 'applied_at', 're...",18,df_application,6048
6,"Index(['id', 'job_id', 'title', 'internal', 'e...",13,df_job_post,261
8,"Index(['id', 'application_id', 'location', 'st...",12,df_scheduled_interview,846
7,"Index(['id', 'name', 'created_at', 'updated_at...",7,df_job_stage,856
1,"Index(['id', 'parent_id', 'name', 'external_id...",6,df_department,24
3,"Index(['id', 'name', 'status', 'created_at', '...",5,df_job,102
9,"Index(['scheduled_interview_id', 'interviewer_...",4,df_scheduled_interviewer,3694
2,"Index(['job_stage_id', 'id', 'name'], dtype='o...",3,df_interview,2242
4,"Index(['application_id', 'job_id', '_fivetran_...",3,df_job_application,14456
5,"Index(['job_id', 'department_id', '_fivetran_s...",3,df_job_department,243


It can be seen from the above that the datasets with the most columns are 'application' and 'job_post'. From this view of the data, there are already a few identifying factors of certain datasets. For example, job_application, job_department and scheduled_interviewer are associative entities that tie together data from different datasets. Most data (exluding associative data) seems to live in the <b>application</b> and <b>interview</b> datasets.

Since the application and interview datasets contain the most data, these two should form the foundation of the models that need to be created.

### Understanding the Relationship Between Datasets: All Tables
The definition (fact/associative, dimension/lookup) of each table may be determined by the number of potential relationships to the table

Please note: Since there are datasets that have been excluded, this needs to be narrowed down to available datasets

In [5]:
# Create relationship stats dataframe
df_dataset_relationships = pd.DataFrame(columns={'Dataset', 'Number of Relationships', 'Relationship Columns'})

# Show possible relations of each dataframe
for index, dataset in enumerate(datasets):
    
    # Filter dataset columns by columns that contain 'id' but are not equal to 'id'
    relationship_columns = list(filter(lambda k: ('id' in k) and (k != 'id'), dataset.columns))
    
    # Append new stats record per dataset
    df_dataset_relationships = df_dataset_relationships.append({'Dataset':dataset_names[index], 'Number of Relationships':len(relationship_columns), 'Relationship Columns': str(relationship_columns)}, ignore_index=True)

    # Test Harness: Print dataset name and relationship columns
    #print(dataset_names[index])
    #print(relationship_columns)
    
# Print sorted dataframe based on number of relationships
df_dataset_relationships.sort_values(by=['Number of Relationships'], ascending=False)

,Relationship Columns,Number of Relationships,Dataset
0,"['candidate_id', 'source_id', 'rejected_reason...",8,df_application
8,"['application_id', 'interview_id', 'organizer_...",3,df_scheduled_interview
9,"['scheduled_interview_id', 'interviewer_id', '...",3,df_scheduled_interviewer
1,"['parent_id', 'external_id']",2,df_department
4,"['application_id', 'job_id']",2,df_job_application
5,"['job_id', 'department_id']",2,df_job_department
2,['job_stage_id'],1,df_interview
6,['job_id'],1,df_job_post
7,['job_id'],1,df_job_stage
3,[],0,df_job


From the above, it can be seen that tables with more than one relationship could be considered associative/fact tables. In a case where there is only one relationship, the deduction is that these table "may be" considered as dimension or lookup tables.

Based on that, the following has been deduced:

- Possible fact/associative tables:
    * df_application
    * df_scheduled_interview
    * df_scheduled_interviewer
    * df_job_application
    * df_job_department
    
- Possible dimension/lookup tables:
    * df_department
    * df_interview
    * df_job_post
    * df_job_stage

- Other:
    * df_job

### Understanding the Relationship Between Datasets: Available Tables
The above relationships are further investigated according to the datasets that have been made available. Since all tables have an 'id' column, some relationships had to be derived.

In [6]:
relationship_list = []

# Iterate through each dataset item in the list. 
# The 'current' item will be the item that all other list items will be compared to in each iteration
for index, current in enumerate(datasets):
  
    current_relationship_list = []
    
    # Iterate through each item in the dataset, to compare to the 'current' item
    for new_index, iterator in enumerate(datasets):
        
        # Construct a preliminary relational column name
        relational_id_name = '{}_id'.format(dataset_names[index].replace('df_',''))
        
        # If relational id name in the columns
        if (relational_id_name in iterator.columns):
                        
            current_relationship_list.append(1)
            
        # Default case, add 0
        else:
            
            current_relationship_list.append(0)
        
    relationship_list.append(current_relationship_list)
        
# Create 'auto-detect' relationship matrix
array = np.array(relationship_list)

df_relationship_matrix = pd.DataFrame(data = array, 
                  index = dataset_names, 
                  columns = dataset_names)

df_relationship_matrix

,df_application,df_department,df_interview,df_job,df_job_application,df_job_department,df_job_post,df_job_stage,df_scheduled_interview,df_scheduled_interviewer
df_application,0,0,0,0,1,0,0,0,1,0
df_department,0,0,0,0,0,1,0,0,0,0
df_interview,0,0,0,0,0,0,0,0,1,0
df_job,0,0,0,0,1,1,1,1,0,0
df_job_application,0,0,0,0,0,0,0,0,0,0
df_job_department,0,0,0,0,0,0,0,0,0,0
df_job_post,0,0,0,0,0,0,0,0,0,0
df_job_stage,0,0,1,0,0,0,0,0,0,0
df_scheduled_interview,0,0,0,0,0,0,0,0,0,1
df_scheduled_interviewer,0,0,0,0,0,0,0,0,0,0


From the above, with special focus on application and interview, the following relationships may be grouped together:
* Application
    * df_application > df_job_application > df_job
    * df_application > df_scheduled_interview > df_interview > df_job_stage > df_job
* Interviews
    * df_interview > df_job_stage > df_job
    * df_interview > df_scheduled_interview > df_scheduled_interview > df_application > df_job_application > df_job

### Understanding the Datasets: Similarities
In a case where data structure merges would be automated, there will be issues when merging tables with the same column headings, representing different data. These similiarities should be handled as 'exceptions' with careful attention on renaming them appropriately before applying merges

In [7]:
df_similiarities = pd.DataFrame(columns={'Current Dataset', 'Iterated Dataset', 'Number of Similarities', 'Similarities'})

# Iterate through each dataset item in the list. 
# The 'current' item will be the item that all other list items will be compared to in each iteration
for index, current in enumerate(datasets):
  
    current_relationship_list = []
    
    # Iterate through each item in the dataset, to compare to the 'current' item
    for new_index, iterator in enumerate(datasets):
        
        # Get the same columns between datasets
        same = np.intersect1d(current.columns, iterator.columns)
        
        # Filter out fivetran results
        same_filtered = list(filter(lambda k: ('fivetran' not in k) and ('id' not in k), same))
        
        if (same_filtered != []) and (dataset_names[index] != dataset_names[new_index]):
            
            # Test Harness: Print number of same fields
            #print('{} - {}'.format(dataset_names[index], dataset_names[new_index]))
            #print(len(same_filtered))
            
            # Append similarity record to the dataframe (excluding 'id' column)
            df_similiarities = df_similiarities.append({'Current Dataset': dataset_names[index], 'Iterated Dataset': dataset_names[new_index], 'Number of Similarities': len(same_filtered), 'Similarities': str(same_filtered)}, ignore_index=True)
            
# Display similiarities dataframe by number of similarities in ascending order
df_similiarities.sort_values(by=['Number of Similarities','Iterated Dataset'], ascending=True)

,Similarities,Current Dataset,Number of Similarities,Iterated Dataset
8,['status'],df_job,1,df_application
22,['status'],df_scheduled_interview,1,df_application
5,['name'],df_interview,1,df_department
9,['name'],df_job,1,df_department
17,['name'],df_job_stage,1,df_department
2,['name'],df_department,1,df_interview
10,['name'],df_job,1,df_interview
18,['name'],df_job_stage,1,df_interview
0,['status'],df_application,1,df_job
3,['name'],df_department,1,df_job


From the above, it can be seen that special attention should be paid to columns like 'status' and 'name' as well as create/update dates when merging datasets

## Transform Datasets

Stage the raw data before performing necessary cleanup tasks and data transformation.

In [8]:
# Applications dataset contains no blanks - data already cleaned - no major data cleanup necessary
# Interviews dataset contains no blanks - data already cleaned - no major data cleanup necessary

### Applications

#### Join application to job_application

In [9]:
# Pre-check to see what the counts are
print(len(df_job_application))
print(len(df_application))

14456
6048


In [10]:
df_application_dataset = df_application
df_application_dataset = pd.merge(df_application_dataset, df_job_application, left_on='id', right_on='application_id', how='left')

# Column Cleanup
df_application_dataset = df_application_dataset.drop(columns = ['_fivetran_synced_x','_fivetran_synced_y'])
df_application_dataset = df_application_dataset.rename(columns={'id': 'application_dataset_id', 'status': 'application_status'})

# Data check: Only 4323 of the 6048 rows have job_id's attached to them
#df_application_dataset[df_application_dataset['job_id'] > 0]

#### Join application_dataset to job

In [11]:
# Pre-check to see what the counts are
print(len(df_application_dataset))
print(len(df_job))

6048
102


In [12]:
df_application_dataset_temp = pd.merge(df_application_dataset, df_job, left_on='job_id', right_on='id', how='left')

# Column Cleanup
df_application_dataset = df_application_dataset_temp.rename(columns={'id': 'job_job_id', 'status': 'job_status', 'name': 'job_name', 'created_at': 'job_created_at', 'closed_at': 'job_closed_at'})

# Data check: Only 3388 of the 6048 rows have job id's attached to them
#df_application_dataset[df_application_dataset['job_job_id'] > 0]

#### Join application_dataset to job_department

In [13]:
# Pre-check to see what the counts are
print(len(df_application_dataset))
print(len(df_job_department))

6048
243


In [14]:
df_application_dataset_temp = pd.merge(df_application_dataset, df_job_department, left_on='job_id', right_on='job_id', how='left')

# Column Cleanup
df_application_dataset = df_application_dataset_temp.rename(columns={'id': 'department_id', 'job_id': 'department_job_id'})

# Data check: Only 4323 of the 6048 rows have job id's attached to them
#df_application_dataset[df_application_dataset['department_id'] > 0]

#### Join application_dataset to department

In [15]:
# Pre-check to see what the counts are
print(len(df_application_dataset))
print(len(df_department))

6048
24


In [16]:
df_application_dataset_temp = pd.merge(df_application_dataset, df_department, left_on='department_id', right_on='id', how='left')

# Column Cleanup
df_application_dataset = df_application_dataset_temp.rename(columns={'id': 'department_department_id', 'name': 'department_name'})

# Data check: Only 4323 of the 6048 rows have job id's attached to them
#df_application_dataset[df_application_dataset['department_department_id'] > 0]

# No departments with parent_id's brought over

#### Join application_dataset to job_stage

In [17]:
# Pre-check to see what the counts are
print(len(df_application_dataset))
print(len(df_job_stage))

6048
856


In [18]:
df_application_dataset_temp = pd.merge(df_application_dataset, df_job_stage, left_on='job_job_id', right_on='job_id', how='left')

# Column Cleanup
df_application_dataset = df_application_dataset_temp.rename(columns={'id': 'job_stage_id', 'name': 'job_stage_name', 'created_at': 'job_stage_created_at', 'updated_at': 'job_stage_updated_at'})
df_application_dataset = df_application_dataset.drop(columns = ['_fivetran_deleted_x','_fivetran_deleted_y'])

df_application_dataset_temp = pd.merge(df_application_dataset, df_job_stage, left_on='current_stage_id', right_on='id', how='left')

# Column Cleanup
df_application_dataset = df_application_dataset_temp.rename(columns={'id': 'application_stage_id', 'name': 'application_stage_name', 'created_at': 'application_stage_created_at', 'updated_at': 'application_stage_updated_at'})

# Data check: There are 29659 of 32267 records with a job stage id
#df_application_dataset[df_application_dataset['job_stage_id'] > 0]

In [19]:
# Post-check to see what the counts are
print(len(df_application_dataset))

32261


In [20]:
# Cleanup column selection
df_application_dataset = df_application_dataset[['candidate_id', 'applied_at', 'rejected_at','last_activity_at', 
                                                 'prospect', 'location_address','application_status', 'is_deleted', 
                                                 'application_id', 'job_name', 'job_status', 'job_created_at','job_closed_at', 
                                                 'job_stage_name','application_stage_name','department_name']]

# All columns that link to datasets that have not been provided (excluding candidate) have been removed

### Interviews

#### Join interview to scheduled_interview

In [21]:
# Pre-check to see what the counts are
print(len(df_interview))
print(len(df_scheduled_interview))

2242
846


In [22]:
df_interview_dataset = df_interview
df_interview_dataset_temp = pd.merge(df_interview, df_scheduled_interview, left_on='id', right_on='interview_id', how='left')

# Column Cleanup
df_interview_dataset = df_interview_dataset_temp.rename(columns={'name':'interview_name', 'id_y': 'scheduled_interview_id', 'id_x': 'interview_interview_id', 'status': 'interview_status', 'created_at': 'interview_created_at','updated_at': 'interview_updated_at', 'end': 'interview_end', 'start': 'interview_start'})

# Data check: Only 820 of the 2242 rows have job_id's attached to them
#df_interview_dataset[df_interview_dataset['scheduled_interview_id'] > 0]

#### Join interview to scheduled_interviewer

In [23]:
# Pre-check to see what the counts are
print(len(df_interview_dataset))
print(len(df_scheduled_interviewer))

2773
3694


In [24]:
df_interview_dataset_temp = pd.merge(df_interview_dataset, df_scheduled_interviewer, left_on='scheduled_interview_id', right_on='scheduled_interview_id', how='left')

# Column Cleanup
df_interview_dataset = df_interview_dataset_temp.rename(columns={'scheduled_interview_id_x': 'scheduled_interview_id', 'scheduled_interview_id_y': 'scheduled_interviewer_id'})

# Data check: Only 1471 of the 2242 rows have job_id's attached to them
#df_interview_dataset[df_interview_dataset['interviewer_id'] > 0]

#### Join interview to job_stage

In [25]:
# Pre-check to see what the counts are
print(len(df_interview_dataset))
print(len(df_job_stage))

3424
856


In [26]:
df_interview_dataset_temp = pd.merge(df_interview_dataset, df_job_stage, left_on='job_stage_id', right_on='id', how='left')

# Column Cleanup
df_interview_dataset = df_interview_dataset_temp.rename(columns={'id': 'job_stage_job_stage_id', 'name': 'job_stage_name', 'created_at': 'job_stage_created_at', 'updated_at': 'job_stage_updated_at'})

# Data check: Only 1824 of the 2242 rows have job_id's attached to them
#df_interview_dataset[df_interview_dataset['job_stage_job_stage_id'] > 0]

#### Join interview to job_department

In [27]:
# Pre-check to see what the counts are
print(len(df_interview_dataset))
print(len(df_job_department))

3424
243


In [28]:
df_interview_dataset_temp = pd.merge(df_interview_dataset, df_job_department, left_on='job_id', right_on='job_id', how='left')

# Column Cleanup
df_interview_dataset = df_interview_dataset_temp.rename(columns={'job_id': 'job_department_job_id'})

# Data check: Only 1824 of the 2242 rows have job_id's attached to them
#df_interview_dataset[df_interview_dataset['department_id'] > 0]

#### Join interview to department

In [29]:
# Pre-check to see what the counts are
print(len(df_interview_dataset))
print(len(df_department))

3424
24


In [30]:
df_interview_dataset_temp = pd.merge(df_interview_dataset, df_department, left_on='department_id', right_on='id', how='left')

# Column Cleanup
df_interview_dataset = df_interview_dataset_temp.rename(columns={'id': 'department_department_id', 'name': 'department_name'})

# Data check: Only 1824 of the 2242 rows have job_id's attached to them
#df_interview_dataset[df_interview_dataset['department_id'] > 0]

#### Join interview to job

In [31]:
# Pre-check to see what the counts are
print(len(df_interview_dataset))
print(len(df_job))

3424
102


In [32]:
df_interview_dataset_temp = pd.merge(df_interview_dataset, df_job, left_on='job_department_job_id', right_on='id', how='left')

# Column Cleanup
df_interview_dataset = df_interview_dataset_temp.rename(columns={'id': 'job_id', 'name': 'job_name', 'status': 'job_status', 'created_at': 'job_created_at', 'closed_at':'job_closed_at'})

# Data check: Only 1815 of the 2242 rows have job_id's attached to them
#df_interview_dataset[df_interview_dataset['job_id'] > 0]

In [33]:
# Post-check to see what the counts are
print(len(df_interview_dataset))

3424


In [34]:
# Cleanup column selection
df_interview_dataset = df_interview_dataset[['interview_interview_id','interview_name','application_id', 'location',
                                             'interview_status','interview_end', 'interview_start', 'interview_id', 
                                             'organizer_id','interviewer_id','job_stage_name',
                                             'department_name', 'job_name','job_status', 'interview_created_at']]

## Output Datasets

Create the required datasets.

In [35]:
# TO DO: Check if path exists, if it doesn't, create the Generated file
file_path += '\Generated'

df_application_dataset.to_csv(r'%s\application.csv' % file_path, index=False)
df_interview_dataset.to_csv(r'%s\interview.csv' % file_path, index=False)

## Visualise Results

The two datasets have been imported into Power BI to be easily visualised according to the questions that were posed, namely:
- Applications:
    - Which jobs have the highest number of applications?
    - What is the conversion rate between the stages of jobs?
    - Which department has created the most job posts?
- Interviews:
    - Which jobs have the highest number of interviews?
    - Which stages in the application have the highest number of interviews?
    - Which interviewers perform the most interviews?
    - Which department has performed the most interviews?
    
The Power BI report and the PDF version of the report (along with all other applicable resources) are available on <a href='http://github.com/JacquiM/Recruitment-Data-Engineering-and-Analysis'>Github</a>

The PDF version of the report is available <a href='http://github.com/JacquiM/Recruitment-Data-Engineering-and-Analysis/blob/main/Report/Final%20Assessment%20Analysis.pdf'>here</a>